In [ ]:
"""
Created on Mon Jul 10 18:30 2023

reformat basic mask to give 0-1-2 and floats for Christoph data
to be used after "format_Christoph_data.sh"

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from cdo import Cdo
import time

In [ ]:
cdo = Cdo()
print('this is CDO version %s'%(cdo.version()))

In [ ]:
nemo_run = 'ctrl94' # 'EPM026','EPM031', 'EPM034'
if nemo_run in ['ctrl94']:
    ystart = 1982
    yend = 2013
elif nemo_run == ['isf94','isfru94']:
    ystart = 2014
    yend = 2100

In [ ]:
inputpath_data='/bettik/burgardc/DATA/SUMMER_PAPER/interim/NEMO_'+nemo_run+'_ANT_STEREO/'

PREPARE THE FLOAT MASK (0 = ocean, 1 = ice shelf, 2 = ground)

In [ ]:
mask_file = xr.open_dataset(inputpath_data+'lsmask_0-1-2_Ant.nc')
file_map_orig = xr.open_dataset(inputpath_data+'mask_variables_of_interest_Ant.nc')
ls_mask012 = mask_file['tmask'].squeeze().astype('float')
ls_mask012.to_dataset(name='ls_mask012').to_netcdf(inputpath_data+'custom_lsmask_Ant.nc')

SEPARATE OTHER VARIABLES

In [ ]:
file_map_orig = xr.open_dataset(inputpath_data+'mask_variables_of_interest_Ant.nc')
isfdraft = file_map_orig['isf_draft'].squeeze()
isfdraft.to_dataset().to_netcdf(inputpath_data+'isfdraft_Ant.nc')

T_levs = file_map_orig['e3w_0'].squeeze()
T_levs_cumsum = T_levs.cumsum(dim='nav_lev') - (T_levs.isel(nav_lev=0)/2)
gdept_0 = T_levs.copy(data=T_levs_cumsum).rename('gdept_0')
    
bathy_metry = file_map_orig[['bathy_metry']].squeeze()
other_mask_vars = xr.merge([gdept_0,bathy_metry])
other_mask_vars.to_netcdf(inputpath_data+'other_mask_vars_Ant.nc')

INTERPOLATE TO STEREOGRAPHIC GRID

In [ ]:
print('interpolate float mask')
time_start = time.time()
cdo.remapbil(inputpath_data+'stereo_grid.nc', input = inputpath_data+'custom_lsmask_Ant.nc', output = inputpath_data+'custom_lsmask_Ant_stereo.nc')
timelength = time.time() - time_start
print(timelength)

In [ ]:
print('interpolate isfdraft')
time_start = time.time()
cdo.remapbil(inputpath_data+'stereo_grid.nc', input = inputpath_data+'isfdraft_Ant.nc', output = inputpath_data+'isfdraft_Ant_stereo.nc')
timelength = time.time() - time_start
print(timelength)

Other variables of interest

In [ ]:
print('computing mask')
time_start = time.time()
cdo.remapbil(inputpath_data+'stereo_grid.nc', input = inputpath_data+'other_mask_vars_Ant.nc', output = inputpath_data+'other_mask_vars_Ant_stereo.nc')
timelength = time.time() - time_start
print(timelength)

In [ ]:
print('creating mean depth coordinate')
time_start = time.time()
cdo.fldmean(input=' -selvar,gdept_0 '+inputpath_data+'other_mask_vars_Ant_stereo.nc', 
              output = inputpath_data+'mask_depth_coord_Ant_stereo.nc') 
timelength = time.time() - time_start
print(timelength)

PREPARE AN ICE SHELF CONCENTRATION

In [ ]:
print('prepare an ice shelf concentration')
float_ls_mask = xr.open_dataset(inputpath_data+'custom_lsmask_Ant_stereo.nc')
isfdraft = xr.open_dataset(inputpath_data+'isfdraft_Ant_stereo.nc')
    
lsmask_012_withcenter = float_ls_mask['ls_mask012'].where(np.isfinite(float_ls_mask['ls_mask012']),2)
should_not_be_ground = (float_ls_mask['ls_mask012']==0) | (isfdraft['isf_draft']>0)
lsmask_below2 = float_ls_mask['ls_mask012'].where(should_not_be_ground,2)

lsmask_below2.to_dataset(name='ls_mask012').to_netcdf(inputpath_data+'custom_lsmask_Ant_stereo_clean.nc')

first_isfdraft_mask = float_ls_mask['ls_mask012'].where(isfdraft['isf_draft']>0,0) # ignore the points of transition between ocean and land
isfdraft_conc = first_isfdraft_mask.where(first_isfdraft_mask <= 1,(2 - first_isfdraft_mask))
isfdraft_conc.to_dataset(name='isfdraft_conc').to_netcdf(inputpath_data+'isfdraft_conc_Ant_stereo.nc')

corrected_isfdraft_raw = isfdraft['isf_draft']/isfdraft_conc
corrected_isfdraft = corrected_isfdraft_raw.where(corrected_isfdraft_raw<=3000,3000).where(corrected_isfdraft_raw>0) # cut unrealistically high drafts
corrections = corrected_isfdraft.to_dataset(name='corrected_isfdraft')

mask_vars = xr.open_dataset(inputpath_data+'other_mask_vars_Ant_stereo.nc')
corrected_bathy_raw = mask_vars['bathy_metry']/isfdraft_conc # not realistically cuttable because bathymetry is set to 0 on land...
corrected_bathy = corrected_bathy_raw.where(corrected_bathy_raw<=3000,3000).where(corrected_bathy_raw>0)

corrections['corrected_isf_bathy'] = corrected_bathy

corrections.to_netcdf(inputpath_data+'corrected_draft_bathy_isf.nc')

FINISHED? THEN MOVE TO "regridding_vars_cdo.ipynb"